In [1]:
# Remove a small sample set 1% for the testing of the final model
# data = df.sample(frac=0.99, random_state=28)
#data_unseen = df.drop(data.index)
#print(f' Data for model: {data.shape},\nData for unseen predictions: {data_unseen.shape}')

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv ('https://raw.githubusercontent.com/pycaret/datasets/main/data/common/concrete.csv')
df.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'strength'],
      dtype='object')

In [8]:
df

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [4]:
df.columns = ['cement','slag','ash','water','superplastic','coarseAgg','fineAgg','age','strength']
df

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [5]:
##df.to_csv('../Regression/', index=False)

In [7]:
df.to_csv('./RegressionData/regressionData-Concrete.csv', index=False)

In [ ]:
## df.to_csv('../../Supervised-Learning/data/concrete.csv', index=False)

In [ ]:
## checking for scaling

def minmax(x):
    return pd.Series(index=['min', 'max','mean','cleanQtile'], data=[x.min(), x.max(),x.mean(),x.quantile(q=0.75) - x.quantile(q=0.25)])
df.apply(minmax).T

,min,max,mean,cleanQtile
cement,102.00,540.0,281.167864,157.625
slag,0.00,359.4,73.895825,142.950
ash,0.00,200.1,54.188350,118.300
water,121.80,247.0,181.567282,27.100
superplastic,0.00,32.2,6.204660,10.200
coarseAgg,801.00,1145.0,972.918932,97.400
fineAgg,594.00,992.6,773.580485,93.050
age,1.00,365.0,45.662136,49.000
strength,2.33,82.6,35.817961,22.425


In [ ]:
## Remove unseen samples

data = df.sample(frac=0.99,random_state=28)
data_unseen = df.drop(data.index)

print(f'Data for model: {data.shape},\nData for unseen predictions: { data_unseen.shape}')

Data for model: (1020, 9),
Data for unseen predictions: (10, 9)


In [ ]:
from pycaret.regression import *
reg = setup(data = data, target='strength', normalize=True, session_id=42)

,Description,Value
0,Session id,42
1,Target,strength
2,Target type,Regression
3,Original data shape,"(1020, 9)"
4,Transformed data shape,"(1020, 9)"
5,Transformed train set shape,"(714, 9)"
6,Transformed test set shape,"(306, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [ ]:
compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:00:55
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,3.4067,24.7720,4.9020,0.9077,0.1499,0.1149,0.0400
et,Extra Trees Regressor,3.4224,29.2213,5.2592,0.8915,0.1576,0.1141,0.0880
rf,Random Forest Regressor,3.9082,32.1600,5.5863,0.8804,0.1736,0.1339,0.0680
gbr,Gradient Boosting Regressor,4.2245,33.1190,5.6632,0.8751,0.1762,0.1446,0.0480
dt,Decision Tree Regressor,4.7547,55.5483,7.3864,0.7939,0.2199,0.1603,0.0250
ada,AdaBoost Regressor,6.4129,62.3452,7.8640,0.7695,0.2798,0.2579,0.0430
knn,K Neighbors Regressor,7.2635,90.5846,9.3506,0.6630,0.3063,0.2691,0.0240
ridge,Ridge Regression,8.1408,107.6078,10.3060,0.6016,0.3307,0.3103,0.0200
br,Bayesian Ridge,8.1556,107.6794,10.3102,0.6014,0.3313,0.3113,0.0230
lr,Linear Regression,8.1338,107.6337,10.3069,0.6014,0.3305,0.3097,0.6450


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,3.4067,24.7720,4.9020,0.9077,0.1499,0.1149,0.0400
et,Extra Trees Regressor,3.4224,29.2213,5.2592,0.8915,0.1576,0.1141,0.0880
rf,Random Forest Regressor,3.9082,32.1600,5.5863,0.8804,0.1736,0.1339,0.0680
gbr,Gradient Boosting Regressor,4.2245,33.1190,5.6632,0.8751,0.1762,0.1446,0.0480
dt,Decision Tree Regressor,4.7547,55.5483,7.3864,0.7939,0.2199,0.1603,0.0250
ada,AdaBoost Regressor,6.4129,62.3452,7.8640,0.7695,0.2798,0.2579,0.0430
knn,K Neighbors Regressor,7.2635,90.5846,9.3506,0.6630,0.3063,0.2691,0.0240
ridge,Ridge Regression,8.1408,107.6078,10.3060,0.6016,0.3307,0.3103,0.0200
br,Bayesian Ridge,8.1556,107.6794,10.3102,0.6014,0.3313,0.3113,0.0230
lr,Linear Regression,8.1338,107.6337,10.3069,0.6014,0.3305,0.3097,0.6450


LGBMRegressor(random_state=42)

In [ ]:
compare_models(exclude=['omp','dummy'])

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:01:11
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,3.4067,24.7720,4.9020,0.9077,0.1499,0.1149,0.0330
et,Extra Trees Regressor,3.4224,29.2213,5.2592,0.8915,0.1576,0.1141,0.0720
rf,Random Forest Regressor,3.9082,32.1600,5.5863,0.8804,0.1736,0.1339,0.0720
gbr,Gradient Boosting Regressor,4.2245,33.1190,5.6632,0.8751,0.1762,0.1446,0.0490
dt,Decision Tree Regressor,4.7547,55.5483,7.3864,0.7939,0.2199,0.1603,0.0270
ada,AdaBoost Regressor,6.4129,62.3452,7.8640,0.7695,0.2798,0.2579,0.0480
knn,K Neighbors Regressor,7.2635,90.5846,9.3506,0.6630,0.3063,0.2691,0.0290
ridge,Ridge Regression,8.1408,107.6078,10.3060,0.6016,0.3307,0.3103,0.0240
lr,Linear Regression,8.1338,107.6337,10.3069,0.6014,0.3305,0.3097,0.0260
br,Bayesian Ridge,8.1556,107.6794,10.3102,0.6014,0.3313,0.3113,0.0250


LGBMRegressor(random_state=42)

In [ ]:
# model is run on the train dataset
model = create_model('lightgbm')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:01:20
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0693,19.7519,4.4443,0.9269,0.1145,0.0855
1,2.5142,14.0463,3.7478,0.9542,0.1049,0.0764
2,3.1267,16.7283,4.0900,0.9300,0.1486,0.1140
3,4.6627,42.0308,6.4831,0.8591,0.1631,0.1371
4,2.9243,17.2832,4.1573,0.9430,0.1334,0.0970
5,3.1628,19.2498,4.3875,0.9142,0.1521,0.1129
6,3.1475,26.0820,5.1071,0.9132,0.1843,0.1280
7,4.2526,36.5330,6.0443,0.8578,0.1549,0.1223
8,3.3503,24.1388,4.9131,0.9212,0.1648,0.1308


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0693,19.7519,4.4443,0.9269,0.1145,0.0855
1,2.5142,14.0463,3.7478,0.9542,0.1049,0.0764
2,3.1267,16.7283,4.0900,0.9300,0.1486,0.1140
3,4.6627,42.0308,6.4831,0.8591,0.1631,0.1371
4,2.9243,17.2832,4.1573,0.9430,0.1334,0.0970
5,3.1628,19.2498,4.3875,0.9142,0.1521,0.1129
6,3.1475,26.0820,5.1071,0.9132,0.1843,0.1280
7,4.2526,36.5330,6.0443,0.8578,0.1549,0.1223
8,3.3503,24.1388,4.9131,0.9212,0.1648,0.1308


In [ ]:
# Training model on test data set
predict_model(model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,3.1752,20.2916,4.5046,0.9277,0.1574,0.1183


,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength,prediction_label
714,254.000000,0.000000,0.000000,198.000000,0.0,968.000000,863.000000,90,26.940001,25.039384
715,230.000000,0.000000,118.300003,195.500000,4.6,1029.400024,758.599976,100,35.340000,33.918469
716,213.800003,98.099998,24.500000,181.699997,6.7,1066.000000,785.500000,100,49.970001,47.179640
717,446.000000,24.000000,79.000000,162.000000,11.6,967.000000,712.000000,7,39.299999,43.824018
718,213.500000,0.000000,174.199997,154.600006,11.7,1052.300049,775.500000,3,17.370001,22.764129
...,...,...,...,...,...,...,...,...,...,...
1015,173.500000,50.099998,173.500000,164.800003,6.5,1006.200012,793.500000,28,38.200001,48.739442
1016,149.000000,236.000000,0.000000,176.000000,13.0,847.000000,893.000000,28,32.959999,31.032593
1017,237.500000,237.500000,0.000000,228.000000,0.0,932.000000,594.000000,90,33.119999,46.150469
1018,475.000000,0.000000,0.000000,228.000000,0.0,932.000000,594.000000,365,41.930000,44.842143


In [ ]:
# Tune model goes agaist train and test 
tuned_model = tune_model(model)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:01:21
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.9476,18.0401,4.2474,0.9333,0.1149,0.0857
1,2.7105,14.5987,3.8208,0.9524,0.1176,0.0854
2,2.5701,13.5404,3.6797,0.9433,0.1268,0.0952
3,4.8620,49.3233,7.0231,0.8347,0.1793,0.1433
4,2.3385,10.5459,3.2474,0.9652,0.1230,0.0888
5,3.0146,18.0419,4.2476,0.9196,0.1406,0.1059
6,2.7017,23.8066,4.8792,0.9208,0.1795,0.1174
7,4.4088,43.0870,6.5641,0.8323,0.1814,0.1324
8,3.2046,18.2958,4.2774,0.9402,0.1517,0.1272


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
print(tuned_model)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=3, feature_fraction=0.8,
              learning_rate=0.2, min_child_samples=6, min_split_gain=0.6,
              num_leaves=30, random_state=42, reg_alpha=0.001, reg_lambda=5)


In [ ]:
final_model = finalize_model(tuned_model)
final_model

Pipeline(memory=FastMemory(location=C:\Users\juud0\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['cement', 'slag', 'ash', 'water',
                                             'superplastic', 'coarseAgg',
                                             'fineAgg', 'age'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('actual_estimator',
                 LGBMRegressor(bagging_fraction=0.8, bagging_freq=3,
                               feature_fraction=0.8, learning_rate=0.2,
                               min_child_samples=6, min_split_gain=0.6,
                               num_leaves=30, random_state=42, reg_alpha=0.001,
                               reg_lambda=5))])

In [ ]:
save_model(final_model, './models/concrete_ltgm')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=C:\Users\juud0\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['cement', 'slag', 'ash', 'water',
                                              'superplastic', 'coarseAgg',
                                              'fineAgg', 'age'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
                 ('actual_estimator',
                  LGBMRegressor(bagging_fraction=0.8, bagging_freq=3,
                                feature_fraction=0.8, learning_rate=0.2,
                                min_child_samples=6, min_split_gain=0.6,
                                num_leaves=30, random_state=42, reg_alpha=0

In [ ]:
model_test = load_model('./models/concrete_ltgm')

Transformation Pipeline and Model Successfully Loaded


In [ ]:
data = [[332.5,	142.5,	0.0,	228.0,	0.0,	932.0,	594.0,	365]]
#41.05

In [ ]:
cols = model_test.feature_names_in_[:-1]
data_df = pd.DataFrame(data, columns=cols)
prediction = model_test.predict(data_df)[0]
prediction

41.35282973394896

In [ ]:
data_unseen

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength
87,286.3,200.9,0.0,144.7,11.2,1004.6,803.7,3,24.40
140,313.3,262.2,0.0,175.5,8.6,1046.9,611.8,56,64.90
152,362.6,189.0,0.0,164.9,11.6,944.7,755.8,56,77.30
192,233.8,0.0,94.6,197.9,4.6,947.0,852.2,56,27.66
453,190.3,0.0,125.2,166.6,9.9,1079.0,798.9,56,31.72
472,446.0,24.0,79.0,162.0,11.6,967.0,712.0,28,44.42
476,446.0,24.0,79.0,162.0,11.6,967.0,712.0,3,25.02
498,355.0,19.0,97.0,145.0,12.3,967.0,871.0,28,55.45
606,236.0,0.0,0.0,194.0,0.0,968.0,885.0,14,12.84
939,145.9,230.5,0.0,202.5,3.4,827.0,871.8,28,32.72


In [ ]:
batch = data_unseen.iloc[:,:-1]
batch
predictions = model_test.predict(batch)
predictions

array([29.16396672, 68.27886247, 77.07838182, 30.60932282, 31.19140918,
       54.39744264, 29.82951451, 46.99453254, 16.57758829, 31.93897401])

In [ ]:
data_unseen['y-hat'] =np.round(predictions,2)
data_unseen['pct_err'] = (abs(data_unseen['strength'] - data_unseen['y-hat'])) / data_unseen['strength']
data_unseen

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength,y-hat,pct_err
87,286.3,200.9,0.0,144.7,11.2,1004.6,803.7,3,24.40,29.16,0.195082
140,313.3,262.2,0.0,175.5,8.6,1046.9,611.8,56,64.90,68.28,0.052080
152,362.6,189.0,0.0,164.9,11.6,944.7,755.8,56,77.30,77.08,0.002846
192,233.8,0.0,94.6,197.9,4.6,947.0,852.2,56,27.66,30.61,0.106652
453,190.3,0.0,125.2,166.6,9.9,1079.0,798.9,56,31.72,31.19,0.016709
472,446.0,24.0,79.0,162.0,11.6,967.0,712.0,28,44.42,54.40,0.224674
476,446.0,24.0,79.0,162.0,11.6,967.0,712.0,3,25.02,29.83,0.192246
498,355.0,19.0,97.0,145.0,12.3,967.0,871.0,28,55.45,46.99,0.152570
606,236.0,0.0,0.0,194.0,0.0,968.0,885.0,14,12.84,16.58,0.291277
939,145.9,230.5,0.0,202.5,3.4,827.0,871.8,28,32.72,31.94,0.023839


In [ ]:
#y is the target and y-hat is the predictions of y on unsendata

In [ ]:
evaluate_model(model_test)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
df

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [ ]:
beta_model = predict_model(model_test,data=df.drop('strength', axis=1))
beta_model

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,prediction_label
0,540.000000,0.000000,0.000000,162.000000,2.5,1040.000000,676.000000,28,75.521229
1,540.000000,0.000000,0.000000,162.000000,2.5,1055.000000,676.000000,28,64.364456
2,332.500000,142.500000,0.000000,228.000000,0.0,932.000000,594.000000,270,40.624615
3,332.500000,142.500000,0.000000,228.000000,0.0,932.000000,594.000000,365,41.352830
4,198.600006,132.399994,0.000000,192.000000,0.0,978.400024,825.500000,360,43.498180
...,...,...,...,...,...,...,...,...,...
1025,276.399994,116.000000,90.300003,179.600006,8.9,870.099976,768.299988,28,43.913968
1026,322.200012,0.000000,115.599998,196.000000,10.4,817.900024,813.400024,28,31.652322
1027,148.500000,139.399994,108.599998,192.699997,6.1,892.400024,780.000000,28,24.214244
1028,159.100006,186.699997,0.000000,175.600006,11.3,989.599976,788.900024,28,32.681527


In [ ]:
beta_model['strength'] = df['strength']
beta_model

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,prediction_label,strength
0,540.000000,0.000000,0.000000,162.000000,2.5,1040.000000,676.000000,28,75.521229,79.99
1,540.000000,0.000000,0.000000,162.000000,2.5,1055.000000,676.000000,28,64.364456,61.89
2,332.500000,142.500000,0.000000,228.000000,0.0,932.000000,594.000000,270,40.624615,40.27
3,332.500000,142.500000,0.000000,228.000000,0.0,932.000000,594.000000,365,41.352830,41.05
4,198.600006,132.399994,0.000000,192.000000,0.0,978.400024,825.500000,360,43.498180,44.30
...,...,...,...,...,...,...,...,...,...,...
1025,276.399994,116.000000,90.300003,179.600006,8.9,870.099976,768.299988,28,43.913968,44.28
1026,322.200012,0.000000,115.599998,196.000000,10.4,817.900024,813.400024,28,31.652322,31.18
1027,148.500000,139.399994,108.599998,192.699997,6.1,892.400024,780.000000,28,24.214244,23.70
1028,159.100006,186.699997,0.000000,175.600006,11.3,989.599976,788.900024,28,32.681527,32.77


In [ ]:
beta_model['err'] = beta_model['strength'] - beta_model['prediction_label']
beta_model

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,prediction_label,strength,err
0,540.000000,0.000000,0.000000,162.000000,2.5,1040.000000,676.000000,28,75.521229,79.99,4.468771
1,540.000000,0.000000,0.000000,162.000000,2.5,1055.000000,676.000000,28,64.364456,61.89,-2.474456
2,332.500000,142.500000,0.000000,228.000000,0.0,932.000000,594.000000,270,40.624615,40.27,-0.354615
3,332.500000,142.500000,0.000000,228.000000,0.0,932.000000,594.000000,365,41.352830,41.05,-0.302830
4,198.600006,132.399994,0.000000,192.000000,0.0,978.400024,825.500000,360,43.498180,44.30,0.801820
...,...,...,...,...,...,...,...,...,...,...,...
1025,276.399994,116.000000,90.300003,179.600006,8.9,870.099976,768.299988,28,43.913968,44.28,0.366032
1026,322.200012,0.000000,115.599998,196.000000,10.4,817.900024,813.400024,28,31.652322,31.18,-0.472322
1027,148.500000,139.399994,108.599998,192.699997,6.1,892.400024,780.000000,28,24.214244,23.70,-0.514244
1028,159.100006,186.699997,0.000000,175.600006,11.3,989.599976,788.900024,28,32.681527,32.77,0.088473


In [ ]:
beta_model['pct_err'] = (abs (beta_model['strength'] - beta_model['prediction_label'])) / beta_model['strength']
beta_model

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,prediction_label,strength,err,pct_err
0,540.000000,0.000000,0.000000,162.000000,2.5,1040.000000,676.000000,28,75.521229,79.99,4.468771,0.055867
1,540.000000,0.000000,0.000000,162.000000,2.5,1055.000000,676.000000,28,64.364456,61.89,-2.474456,0.039982
2,332.500000,142.500000,0.000000,228.000000,0.0,932.000000,594.000000,270,40.624615,40.27,-0.354615,0.008806
3,332.500000,142.500000,0.000000,228.000000,0.0,932.000000,594.000000,365,41.352830,41.05,-0.302830,0.007377
4,198.600006,132.399994,0.000000,192.000000,0.0,978.400024,825.500000,360,43.498180,44.30,0.801820,0.018100
...,...,...,...,...,...,...,...,...,...,...,...,...
1025,276.399994,116.000000,90.300003,179.600006,8.9,870.099976,768.299988,28,43.913968,44.28,0.366032,0.008266
1026,322.200012,0.000000,115.599998,196.000000,10.4,817.900024,813.400024,28,31.652322,31.18,-0.472322,0.015148
1027,148.500000,139.399994,108.599998,192.699997,6.1,892.400024,780.000000,28,24.214244,23.70,-0.514244,0.021698
1028,159.100006,186.699997,0.000000,175.600006,11.3,989.599976,788.900024,28,32.681527,32.77,0.088473,0.002700


In [ ]:
beta_model = beta_model.sort_values('pct_err', ascending=False)
beta_model

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,prediction_label,strength,err,pct_err
114,362.600006,189.000000,0.0,164.899994,11.6,944.700012,755.799988,7,46.291124,22.90,-23.391124,1.021446e+00
763,385.000000,0.000000,0.0,186.000000,0.0,966.000000,763.000000,1,12.170906,6.27,-5.900906,9.411333e-01
732,349.000000,0.000000,0.0,192.000000,0.0,1056.000000,809.000000,7,15.355306,9.01,-6.345306,7.042515e-01
746,500.000000,0.000000,0.0,200.000000,0.0,1125.000000,613.000000,1,20.823561,12.64,-8.183561,6.474336e-01
688,108.300003,162.399994,0.0,203.500000,0.0,938.200012,849.000000,3,3.271568,2.33,-0.941568,4.041065e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
729,331.000000,0.000000,0.0,192.000000,0.0,1025.000000,821.000000,90,37.913233,37.91,-0.003233,8.527886e-05
646,250.199997,166.800003,0.0,203.500000,0.0,977.599976,694.099976,90,47.711451,47.71,-0.001451,3.042196e-05
951,152.699997,144.699997,0.0,178.100006,8.0,999.700012,822.200012,28,19.010258,19.01,-0.000258,1.355663e-05
388,385.000000,0.000000,136.0,158.000000,20.0,903.000000,768.000000,28,55.550396,55.55,-0.000396,7.121914e-06


In [ ]:
beta_model[beta_model['pct_err'] > 0.15].shape

(32, 12)

In [ ]:
beta_model[beta_model['pct_err'] > 0.15]

,cement,slag,ash,water,superplastic,coarseAgg,fineAgg,age,prediction_label,strength,err,pct_err
114,362.600006,189.000000,0.000000,164.899994,11.6,944.700012,755.799988,7,46.291124,22.90,-23.391124,1.021446
763,385.000000,0.000000,0.000000,186.000000,0.0,966.000000,763.000000,1,12.170906,6.27,-5.900906,0.941133
732,349.000000,0.000000,0.000000,192.000000,0.0,1056.000000,809.000000,7,15.355306,9.01,-6.345306,0.704251
746,500.000000,0.000000,0.000000,200.000000,0.0,1125.000000,613.000000,1,20.823561,12.64,-8.183561,0.647434
688,108.300003,162.399994,0.000000,203.500000,0.0,938.200012,849.000000,3,3.271568,2.33,-0.941568,0.404107
235,213.800003,98.099998,24.500000,181.699997,6.7,1066.000000,785.500000,14,23.821200,17.84,-5.981200,0.335269
294,168.899994,42.200001,124.300003,158.300003,10.8,1080.800049,796.200012,3,9.660078,7.40,-2.260078,0.305416
606,236.000000,0.000000,0.000000,194.000000,0.0,968.000000,885.000000,14,16.577588,12.84,-3.737588,0.291089
477,446.000000,24.000000,79.000000,162.000000,11.6,967.000000,712.000000,3,29.829515,23.35,-6.479515,0.277495
289,182.000000,45.200001,122.000000,170.199997,8.2,1059.400024,780.700012,3,9.317651,7.32,-1.997651,0.272903


In [ ]:
print(df.shape,'\n', beta_model[beta_model['pct_err']> 0.15].shape)

(1030, 9) 
 (32, 12)


In [ ]:
print(beta_model[beta_model['pct_err']> 0.15].shape[0]/df.shape[0])

0.031067961165048542
